In [1]:
import numpy as np 
import pandas as pd
import re
import gc
import os
print(os.listdir("../input"))
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

['test_tOlRoBf.csv', 'train_F3WbcTw.csv', 'sample_submission_i5xnIZD.csv']


Using TensorFlow backend.


In [2]:
train = pd.read_csv("../input/train_F3WbcTw.csv")
test = pd.read_csv("../input/test_tOlRoBf.csv")

In [3]:
train['text'] = train['text'] + "This observation is for " + train['drug']
test['text'] = test['text'] + "This observation is for " + test['drug']

In [4]:
import re
def pre_process(text):
    new_text =re.sub('[0-9]', '', text)
    new_text = re.sub(r"\u200b","",new_text)
    new_text = re.sub(r"\.+",".",new_text)
    new_text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '',new_text, flags=re.MULTILINE)
    new_text = re.sub("'", "", new_text)
    new_text = re.sub(r'↑', '', new_text)
    new_text = re.sub("\t", "", new_text)
    new_text = re.sub("\xa0", "", new_text)
    new_text = re.sub("\(|\)|\[|\]", "", new_text)
    new_text = re.sub("\n", "", new_text)
    new_text = re.sub("\.", "", new_text)
    new_text = re.sub("\,", " ", new_text)
    new_text = re.sub("[/%]", " ", new_text)
    new_text = re.sub('[/%:;]', '', new_text)
    new_text = re.sub(' +', ' ', new_text)
    return new_text

In [5]:
%%time

# remove URL's from train and test
for index, row in train['text'].iteritems():
    train['text'][index] = pre_process(row)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


CPU times: user 6min 54s, sys: 128 ms, total: 6min 54s
Wall time: 6min 55s


In [6]:
%%time 

for index, row in test['text'].iteritems():
    test['text'][index] = pre_process(row)

CPU times: user 2.2 s, sys: 20 ms, total: 2.22 s
Wall time: 2.19 s


In [7]:
# Handle Won't 

replaceWords1 = { "won't":"will not","$&@*#":"in most profane vulgar shitty terms","#$&@*#":"shitty",
 "can't":"cannot","aren't": 'are not',
 "Aren't": 'Are not',
 "AREN'T": 'ARE NOT',
 "C'est": "C'est",
 "C'mon": "C'mon",
 "c'mon": "c'mon",
 "can't": 'cannot',
 "Can't": 'Cannot',
 "CAN'T": 'CANNOT',
 "con't": 'continued',
 "cont'd": 'continued',
 "could've": 'could have',
 "couldn't": 'could not',
 "Couldn't": 'Could not',
 "didn't": 'did not',
 "Didn't": 'Did not',
 "DIDN'T": 'DID NOT',
 "don't": 'do not',
 "Don't": 'Do not',
 "DON'T": 'DO NOT',
 "doesn't": 'does not',
 "Doesn't": 'Does not',
 "else's": 'else',
 "gov's": 'government',
 "Gov's": 'government',
 "gov't": 'government',
 "Gov't": 'government',
 "govt's": 'government',
 "gov'ts": 'governments',
 "hadn't": 'had not',
 "hasn't": 'has not',
 "Hasn't": 'Has not',
 "haven't": 'have not',
 "Haven't": 'Have not',
 "he's": 'he is',
 "He's": 'He is',
 "he'll": 'he will',
 "He'll": 'He will',
 "he'd": 'he would',
 "He'd": 'He would',
 "Here's": 'Here is',
 "here's": 'here is',
 "I'm": 'I am',
 "i'm": 'i am',
 "I'M": 'I am',
 "I've": 'I have',
 "i've": 'i have',
 "I'll": 'I will',
 "i'll": 'i will',
 "I'd": 'I would',
 "i'd": 'i would',
 "ain't": 'is not',
 "isn't": 'is not',
 "Isn't": 'Is not',
 "ISN'T": 'IS NOT',
 "it's": 'it is',
 "It's": 'It is',
 "IT'S": 'IT IS',
 "I's": 'It is',
 "i's": 'it is',
 "it'll": 'it will',
 "It'll": 'It will',
 "it'd": 'it would',
 "It'd": 'It would',
 "Let's": "Let's",
 "let's": 'let us',
 "ma'am": 'madam',
 "Ma'am": "Madam",
 "she's": 'she is',
 "She's": 'She is',
 "she'll": 'she will',
 "She'll": 'She will',
 "she'd": 'she would',
 "She'd": 'She would',
 "shouldn't": 'should not',
 "that's": 'that is',
 "That's": 'That is',
 "THAT'S": 'THAT IS',
 "THAT's": 'THAT IS',
 "that'll": 'that will',
 "That'll": 'That will',
 "there's": 'there is',
 "There's": 'There is',
 "there'll": 'there will',
 "There'll": 'There will',
 "there'd": 'there would',
 "they're": 'they are',
 "They're": 'They are',
 "they've": 'they have',
 "They've": 'They Have',
 "they'll": 'they will',
 "They'll": 'They will',
 "they'd": 'they would',
 "They'd": 'They would',
 "wasn't": 'was not',
 "we're": 'we are',
 "We're": 'We are',
 "we've": 'we have',
 "We've": 'We have',
 "we'll": 'we will',
 "We'll": 'We will',
 "we'd": 'we would',
 "We'd": 'We would',
 "What'll": 'What will',
 "weren't": 'were not',
 "Weren't": 'Were not',
 "what's": 'what is',
 "What's": 'What is',
 "When's": 'When is',
 "Where's": 'Where is',
 "where's": 'where is',
 "Where'd": 'Where would',
 "who're": 'who are',
 "who've": 'who have',
 "who's": 'who is',
 "Who's": 'Who is',
 "who'll": 'who will',
 "who'd": 'Who would',
 "Who'd": 'Who would',
 "won't": 'will not',
 "Won't": 'will not',
 "WON'T": 'WILL NOT',
 "would've": 'would have',
 "wouldn't": 'would not',
 "Wouldn't": 'Would not',
 "would't": 'would not',
 "Would't": 'Would not',
 "y'all": 'you all',
 "Y'all": 'You all',
 "you're": 'you are',
 "You're": 'You are',
 "YOU'RE": 'YOU ARE',
 "you've": 'you have',
 "You've": 'You have',
 "y'know": 'you know',
 "Y'know": 'You know',
 "ya'll": 'you will',
 "you'll": 'you will',
 "You'll": 'You will',
 "you'd": 'you would',
 "You'd": 'You would',
 "Y'got": 'You got',
 'cause': 'because',
 "had'nt": 'had not',
 "Had'nt": 'Had not',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how'll": 'how will',
 "how's": 'how is',
 "I'd've": 'I would have',
 "I'll've": 'I will have',
 "i'd've": 'i would have',
 "i'll've": 'i will have',
 "it'd've": 'it would have',
 "it'll've": 'it will have',
 "mayn't": 'may not',
 "might've": 'might have',
 "mightn't": 'might not',
 "mightn't've": 'might not have',
 "must've": 'must have',
 "mustn't": 'must not',
 "mustn't've": 'must not have',
 "needn't": 'need not',
 "needn't've": 'need not have',
 "o'clock": 'of the clock',
 "oughtn't": 'ought not',
 "oughtn't've": 'ought not have',
 "shan't": 'shall not',
 "sha'n't": 'shall not',
 "shan't've": 'shall not have',
 "she'd've": 'she would have',
 "she'll've": 'she will have',
 "should've": 'should have',
 "shouldn't've": 'should not have',
 "so've": 'so have',
 "so's": 'so as',
 "this's": 'this is',
 "that'd": 'that would',
 "that'd've": 'that would have',
 "there'd've": 'there would have',
 "they'd've": 'they would have',
 "they'll've": 'they will have',
 "to've": 'to have',
 "we'd've": 'we would have',
 "we'll've": 'we will have',
 "what'll": 'what will',
 "what'll've": 'what will have',
 "what're": 'what are',
 "what've": 'what have',
 "when's": 'when is',
 "when've": 'when have',
 "where'd": 'where did',
 "where've": 'where have',
 "who'll've": 'who will have',
 "why's": 'why is',
 "why've": 'why have',
 "will've": 'will have',
 "won't've": 'will not have',
 "wouldn't've": 'would not have',
 "y'all'd": 'you all would',
 "y'all'd've": 'you all would have',
 "y'all're": 'you all are',
 "y'all've": 'you all have',
 "you'd've": 'you would have',
 "you'll've": 'you will have',
'bebecause':'be because',
'I’m':'I am',
              'it’s':'it is',
                 'I’ve':'I have',
                 'don’t':'do not',
                'However':'but',
                 'It’s':'It is',
                 'didn’t':'did not',
                 'can’t':'can not',
                 'that’s':'that is',
'doesn’t':'does not',
'I’d':'I had',
'isn’t':'is not',
'wasn’t':'was not'
                
                }

def wordreplace(tweet,replaceWords):
    for key in replaceWords:
        tweet = tweet.replace(key,replaceWords[key])
    return tweet

#for index, row in train['text'].iteritems():
#    train['text'][index] = wordreplace(row,replaceWords1)
    
#for index, row in test['text'].iteritems():
#    test['text'][index] = wordreplace(row,replaceWords1)

In [8]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

--2019-07-28 18:00:22--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   127MB/s    in 3.2s    

2019-07-28 18:00:26 (120 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

--2019-07-28 18:00:26--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/

In [9]:
import modeling
import optimization
import run_classifier
import tokenization

In [10]:
folder = 'model_folder'
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)

In [11]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/outputs'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>>  BERT pretrained directory: {BERT_PRETRAINED_DIR}')

>> Model output directory: model_folder/outputs
>>  BERT pretrained directory: model_folder/uncased_L-12_H-768_A-12


In [12]:
X = train['text']
y = train['sentiment']
test_pred = test['text']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify = y.values , test_size=0.2, random_state=42)

In [14]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 30
EVAL_BATCH_SIZE = 20
LEARNING_RATE = 1e-5 #ncreasing the LR
NUM_TRAIN_EPOCHS = 35.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 200

# Model configs
SAVE_CHECKPOINTS_STEPS = 100000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(8)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

In [15]:
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

Please wait...
>> Started training at 2019-07-28 18:01:14.635404 
  Num examples = 4223
  Batch size = 30
>> Finished training at 2019-07-28 19:07:21.181855


In [16]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 500

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [17]:
predict_examples = create_examples(X_test, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

In [18]:
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

{}


In [19]:
from sklearn.metrics import accuracy_score

In [20]:
print("Accuracy of BERT is:",accuracy_score(y_test,preds))

Accuracy of BERT is: 0.6979166666666666


In [21]:
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.35      0.41      0.38       123
           1       0.47      0.42      0.45       168
           2       0.81      0.80      0.81       765

    accuracy                           0.70      1056
   macro avg       0.54      0.55      0.54      1056
weighted avg       0.70      0.70      0.70      1056



In [22]:
predict_examples = create_examples(test['text'], 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

In [23]:
preds_test = []
for prediction in result:
      preds_test.append(np.argmax(prediction['probabilities']))

{}


In [24]:
sub = pd.read_csv("../input/sample_submission_i5xnIZD.csv")
sub['sentiment'] = preds_test
sub.to_csv("ELMO_1L.csv",index=False)

In [25]:
preds_test = []
for prediction in result:
      preds_test.append(prediction['probabilities'])

In [26]:
avg_BERT = pd.DataFrame(preds_test)
avg_BERT.to_csv("BERT_prob.csv",index=False)